# Build Repurchase Dataset

The purpose of this notebook is to **create weekly datasets to predict the likelihood that an article will be repurchased within a 7 day window**. 

The training set for this model is **all articles that a customer has purchased up until the 7-day window in question**.

This model uses the following features:
- How many days since the article was last purchased by the customer?
- How many times has the article been purchased by the customer?
- How often does the customer repurchase articles? (Total number of purchases / Total number of unique purchases)
- How many days since the article was first sold + last sold at H&M?
- How often is the article repurchased? (Total number of purchases / total number of unique customers purchased)
- What is the median age of the article purchaser? How far off is the median age from the customer in question?
- How many weeks removed are we from the peak sales week of the given article?
- How big was their cart? (E.g. how many articles did they purchase that day)
- What was the original purchase price?
- Are there any current sales for the article? (Average price sold last week / Average price sold overall)
- Tf-idf PCA features against the article descriptions
- How similar is this article's metadata to previous purchases? (Color, index group, garment group)
- How popular is the article last week/month? (average sales per week/month vs sales last week/month)

## Import statements

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import datetime as dt

pd.set_option('display.max_columns', 500)
pd.set_option('display.max_rows', 500)

from scipy import sparse 
from pandas.api.types import CategoricalDtype 

from sklearn.neighbors import NearestNeighbors
from scipy.spatial import KDTree

from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix,roc_curve,roc_auc_score,f1_score,precision_score,recall_score
from sklearn.model_selection import GridSearchCV,GroupKFold
from sklearn.calibration import CalibratedClassifierCV

import xgboost as xgb

# import nltk

# from nltk import *
# nltk.download('stopwords')
# nltk.download('punkt')
# from nltk.corpus import stopwords

from sklearn.feature_extraction import text
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import PCA

from tqdm import tqdm
tqdm.pandas()

## Read in data + fix data types

In [ ]:
sample = '_05'

# Read in articles data
df_art = pd.read_csv('../Data/articles/articles'+sample+'.csv')
df_cust = pd.read_csv('../Data/customers/customers'+sample+'.csv')
df_trans = pd.read_csv('../Data/transactions_train/transactions_train'+sample+'.csv')

In [ ]:
# Fix format of article IDs
df_art['article_id'] = df_art['article_id'].astype(str).str.zfill(10)
df_art['detail_desc'] = df_art['detail_desc'].astype(str)
df_trans['article_id'] = df_trans['article_id'].astype(str).str.zfill(10)

# Fix datetime type
df_trans['t_dat'] = pd.to_datetime(df_trans['t_dat'])

# Build df_cust age brackets
df_cust['Age_Bracket'] = pd.cut(df_cust['age'],[1,19,29,39,49,59,200],labels=[1,2,3,4,5,6]).fillna(2)

# Update the color column for df_art
df_art['color'] = np.where(df_art['perceived_colour_master_name'].isin(['Blue','Turquoise','Bluish Green']),'Blue',\
                  np.where(df_art['perceived_colour_master_name'].isin(['Green','Yellowish Green','Khaki green']),'Green',\
                  np.where(df_art['perceived_colour_master_name'].isin(['Brown','Beige','Mole']),'Brown',\
                  np.where(df_art['perceived_colour_master_name'].isin(['Grey','Metal']),'Grey',\
                           df_art['perceived_colour_master_name']))))

# Build General Pred

In [ ]:
def get_general_pred(dfx):
    
    df_build = dfx.copy()
    
    last_ts = df_build['t_dat'].max()
    last_day = last_ts.strftime('%Y-%m-%d')

    df_build['subdays'] = (last_ts - df_build['t_dat'])
    df_build['temp'] = df_build['subdays'].dt.floor('7D')
    df_build['ldbw'] = last_ts - df_build['temp']

    del df_build['subdays']
    del df_build['temp']
    
    weekly_sales = df_build.drop('customer_id', axis=1).groupby(['ldbw', 'article_id']).count().reset_index()
    weekly_sales = weekly_sales.rename(columns={'t_dat': 'count'})
    weekly_sales = weekly_sales[['ldbw','article_id','count']].copy()

    df_build = pd.merge(df_build,weekly_sales, on=['ldbw', 'article_id'])
    weekly_sales = weekly_sales.reset_index().set_index('article_id')
    
    df = pd.merge(df_build,
        weekly_sales.loc[weekly_sales['ldbw']==last_day, ['count']],
        on='article_id', suffixes=('','_targ'))

    df['count_targ'].fillna(0, inplace=True)
    df['quotient'] = df['count_targ'] / df['count']
    
    target_sales = df.drop('customer_id', axis=1).groupby('article_id')['quotient'].sum()
    
    return target_sales

# Build the dataset - completed features

In [ ]:
def build_dataset(date1,date2,generate_test=True):
    '''
    Potential ideas for new features:
    - How big was the cart when this item was most recently purchased?
    - How many days since the customer FIRST purchased the article (currently only have most recent)
    - Monetary information
    '''
    
    # Create the training datasets
    df_avail = df_trans.loc[df_trans['t_dat'] < date1].copy()
    df_unique = df_avail.groupby(['customer_id','article_id','t_dat']).agg({'price':'mean'}).reset_index()
    df_lastpurchase = df_avail.groupby(['customer_id','article_id']).agg({'t_dat':'max'}).reset_index()
    df_baseline = df_avail.groupby(['customer_id','article_id']).agg({'t_dat':'max'}).reset_index()
    
    
    # Response variable: was the article purchased in the 7 days including/after the threshold date
    print('Step 1: response variable')
    if generate_test:
        df_test = df_trans.loc[(df_trans['t_dat'] >= date1)&(df_trans['t_dat'] <= date2),\
                               ['customer_id','article_id']].copy().drop_duplicates()
        df_test['Response'] = 1
        df_baseline = pd.merge(df_baseline,df_test,how='left',on=['customer_id','article_id']).fillna(0)

        
    # Find number of days since last time this article was purchased by this customer
    print('Step 2: days since last purchase')
    df_baseline['thres'] = date1
    df_baseline['thres'] = pd.to_datetime(df_baseline['thres'])
    df_baseline['DaysSinceLastPurchased'] = (df_baseline['thres'] - df_baseline['t_dat']).dt.days
    del df_baseline['thres']
    
    
    # Total number of times customer X purchased article Y
    print('Step 3: num times purchased')
    df_num_times = df_avail.groupby(['customer_id','article_id']).size().reset_index().rename(\
                                                                                columns={0:'NumTimesCustPurchasedArt'})
    df_baseline = pd.merge(df_baseline,df_num_times,how='left',on=['customer_id','article_id']).fillna(0)
    del df_num_times
    
    
    # What percent of articles are returned - customer repurchase factor
    print('Step 4: percent of articles repurchased')
    df_returns = df_unique.groupby('customer_id').agg({'article_id':['count','nunique']}).reset_index()
    df_returns.columns = ['customer_id','num_articles','num_unique']
    df_returns['RepurchaseFactor_cust'] = df_returns['num_articles'] / df_returns['num_unique']
    df_baseline = pd.merge(df_baseline,df_returns[['customer_id','RepurchaseFactor_cust']],\
                           how='left',on='customer_id')
    df_baseline['RepurchaseFactor_cust'] = df_baseline['RepurchaseFactor_cust'].fillna(1)
    
    
    # How many days since the article was first sold at H&M?
    print('Step 5: days since article was first/last sold at H&M')
    df_sold = df_avail.groupby('article_id').agg({'t_dat':['min','max']}).reset_index()
    df_sold.columns = ['article_id','FirstSold','LastSold']
    df_sold['DaysSinceFirstSold'] = (dt.datetime.strptime(date1,'%Y-%m-%d') - df_sold['FirstSold']).dt.days
    df_sold['DaysSinceLastSold'] = (dt.datetime.strptime(date1,'%Y-%m-%d') - df_sold['LastSold']).dt.days
    df_baseline = pd.merge(df_baseline,df_sold[['article_id','DaysSinceFirstSold','DaysSinceLastSold']],\
                                                   how='left',on='article_id').fillna(0)
    
    del df_sold
    
    
    # Article repurchase factor
    print('Step 6: article repurchase factor')
    df_art_rep =df_unique.groupby('article_id').agg({'customer_id':['count','nunique']}).reset_index()
    df_art_rep.columns = ['article_id','num_cust','num_unique']
    df_art_rep['RepurchaseFactor_art'] = df_art_rep['num_cust'] / df_art_rep['num_unique']
    df_baseline = pd.merge(df_baseline,df_art_rep[['article_id','RepurchaseFactor_art']],\
                           how='left',on='article_id')
    df_baseline['RepurchaseFactor_art'] = df_baseline['RepurchaseFactor_art'].fillna(1)
    
    
    # Age of customer + subscription status
    print('Step 7: age, median age of article purchasers')
    df_baseline = pd.merge(df_baseline,df_cust[['customer_id','age']],how='left',on='customer_id')
    df_baseline['age'] = df_baseline['age'].fillna(32)
    
    # What is the median age of the article purchasers?
    df_lastpurchase = pd.merge(df_lastpurchase,df_cust[['customer_id','age']],how='left',on='customer_id').fillna(32)
    df_midage = df_lastpurchase.groupby('article_id').agg({'age':'median'}).reset_index().rename(\
                                                                                columns={'age':'MedianAge'})
    df_baseline = pd.merge(df_baseline,df_midage,how='left',on='article_id').fillna(32)
    df_baseline['YearsFromMedianAge'] = df_baseline['age'] - df_baseline['MedianAge']
    del df_midage
    del df_lastpurchase
    
    
    # How far removed from the article's peak?
    print('Step 8: how far from customers peak?')
    test_week = dt.datetime.strptime(date1,'%Y-%m-%d').isocalendar()[1]

    df_avail['weekNum'] = df_avail.t_dat.dt.isocalendar().week
    df_week_count = df_avail.groupby(['article_id','weekNum']).size().reset_index()
    df_week_count['rank'] = df_week_count.groupby('article_id')[0].rank('first',ascending=False)
    df_week_count = df_week_count.loc[df_week_count['rank']==1,['article_id','weekNum']]

    df_baseline = pd.merge(df_baseline,df_week_count,how='left',on='article_id')

    df_baseline['TestWeekNum'] = test_week
    df_baseline['Try1'] = (df_baseline['weekNum'] - df_baseline['TestWeekNum']).abs()
    df_baseline['Try2'] = (52 + df_baseline['weekNum'] - df_baseline['TestWeekNum']).abs()
    df_baseline['WeeksFromPeak'] = df_baseline[['Try1','Try2']].min(axis=1)
    del df_baseline['weekNum']
    del df_baseline['TestWeekNum']
    del df_baseline['Try1']
    del df_baseline['Try2']
    
    
    # How many items did they purchase that day?
    print('Step 9: what was the size of their cart?')
    df_cart_size = df_avail.groupby(['customer_id','article_id','t_dat']).size().reset_index()
    df_cart_size.columns = ['customer_id','article_id','t_dat','CartSize']

    df_baseline = pd.merge(df_baseline,df_cart_size,how='left',on=['customer_id','article_id','t_dat'])
    del df_cart_size
    
    
    # What was the most recent purchase price?
    print('Step 10: add original purchase price into the mix')
    df_baseline = pd.merge(df_baseline,df_unique,how='left',on=['customer_id','article_id','t_dat'])
    
    
    # Add feature identifying potential discounts!
    print('Step 11: Discounts')
    weekBeforeStart = dt.datetime.strptime(date1,'%Y-%m-%d') - dt.timedelta(days=7)
    weekBeforeEnd = dt.datetime.strptime(date1,'%Y-%m-%d') - dt.timedelta(days=1)
    
    df_mean_price = df_avail.loc[df_avail['t_dat'] <= weekBeforeEnd].groupby('article_id').agg({'price':'mean'})

    trainPurchases = df_avail.loc[(df_avail['t_dat'] <= weekBeforeEnd)&\
                                        (df_avail['t_dat'] >= weekBeforeStart)].groupby('article_id').agg({'price':'mean'})

    trainPurchases = pd.merge(trainPurchases,df_mean_price,left_index=True,right_index=True)
    trainPurchases['priceScaler'] = trainPurchases['price_x'] / trainPurchases['price_y']
    trainPurchases = trainPurchases[['priceScaler']].copy().reset_index()

    df_baseline = pd.merge(df_baseline,trainPurchases,how='left',on='article_id').fillna(1)
    del df_mean_price
    del trainPurchases
    
    
    print('Step 12: Append PCA')
    pca = pd.read_csv('../Datasets/PCA_Vectorizer.csv')
    pca['article_id'] = pca['article_id'].astype(str).str.zfill(10)
    df_baseline = pd.merge(df_baseline,pca[['article_id','PCA1','PCA2','PCA3','PCA4','PCA5']],on='article_id')
    del pca
    
    
    # Compare article metadata to customer historical metadata purchases
    print('Step 13: how similar is this product metadata to previous purchases')
    df_dummies = pd.get_dummies(df_art[['product_group_name','perceived_colour_value_name','color','index_code','garment_group_name']])
    df_dummies.index = df_art['article_id']
    df_dummies = df_dummies[[i for i in list(df_dummies.columns) if 'Unknown' not in i or 'Undefined' not in i or \
                            'undefined' not in i]]
    df_dummies = df_dummies.loc[:,df_dummies.sum() > 500].reset_index()

    df_trans_dummy = pd.merge(df_avail[['customer_id','article_id']].drop_duplicates(),df_dummies,on='article_id')
    del df_dummies

    df_groups = df_trans_dummy[[i for i in df_trans_dummy.columns if i not in \
                                ['t_dat','price','sales_channel_id']]].groupby('customer_id').sum()
    df_num_purchases = pd.DataFrame(df_trans_dummy.groupby('customer_id').size()).rename(\
                                                                columns={0:'num_purchases'}).reset_index()
    df_groups = pd.merge(df_groups,df_num_purchases,on='customer_id')
    del df_num_purchases

    for col in [i for i in df_groups.columns if i not in ['customer_id','num_purchases']]:
        df_groups[col] = df_groups[col] / df_groups['num_purchases']
    del df_groups['num_purchases']

    df_trans_full = pd.merge(df_trans_dummy,df_groups,on='customer_id',suffixes = ('','_cust'))
    df_trans_join = df_trans_full[['customer_id','article_id']].copy()
    del df_trans_dummy
    del df_groups

    for colType in ['product_group_name','perceived_colour_value_name','color','index_code','garment_group_name']:
        df_trans_join[colType+'_similarity'] = 0
        for col in [j for j in df_trans_full.columns if j[:len(colType)] == colType and j[-5:] != '_cust']:
            df_trans_join[colType+'_similarity'] += (df_trans_full[col] * df_trans_full[col + '_cust'])
    df_trans_join['overall_metadata_similarity'] = df_trans_join.iloc[:,2:].sum(axis=1)

    del df_trans_full
    df_baseline = pd.merge(df_baseline,df_trans_join,how='left',on=['customer_id','article_id']).fillna(0)
    del df_trans_join
    
    
    # Popularity measure from online metric
    print('Step 14: new popularity measure')
    df_pop = pd.DataFrame(get_general_pred(df_avail)).reset_index()
    df_pop.columns = ['article_id','popularity_quotient']
    df_baseline = pd.merge(df_baseline,df_pop,how='left',on='article_id').fillna(0)
    del df_pop
    
    
    # What is the weekly/monthly/overall popularity of the article?
    print('Step 15: Average weekly article sales, last week article sales, ratio')
    df_avail['year'] = df_avail['t_dat'].dt.year
    df_avail['week'] = df_avail['t_dat'].dt.isocalendar().week

    num_weeks = (dt.datetime.strptime(date1,'%Y-%m-%d') - dt.datetime(2018,9,23)).days / 7

    df_weekly = df_avail.groupby(['article_id','year','week']).size().reset_index().rename(columns={0:'count'})
    del df_avail['week']
    del df_avail['year']
    
    df_avg = df_weekly.groupby('article_id').agg({'count':'sum'}).reset_index()
    df_avg.columns = ['article_id','PurchaseRatePerWeek']
    df_avg['PurchaseRatePerWeek'] = df_avg['PurchaseRatePerWeek'] / num_weeks
    df_baseline = pd.merge(df_baseline,df_avg,how='left',on='article_id').fillna(0)
    del df_weekly
    del df_avg
    
    df_avail_last_week = df_avail.loc[df_avail['t_dat'] >= \
                            (dt.datetime.strptime(date1,'%Y-%m-%d') - dt.timedelta(days=7))].copy()
    df_avail_last_week = df_avail_last_week.groupby('article_id').size().reset_index().rename(\
                                                                    columns={0:'PurchaseRateLastWeek'})
    df_baseline = pd.merge(df_baseline,df_avail_last_week,on='article_id',how='left').fillna(0)
    df_baseline['LastWeekPopularity'] = np.where(df_baseline['PurchaseRatePerWeek'] == 0,0,\
                                    df_baseline['PurchaseRateLastWeek']/df_baseline['PurchaseRatePerWeek'])
    del df_avail_last_week
    
    df_avail_last_month = df_avail.loc[df_avail['t_dat'] >= \
                            (dt.datetime.strptime(date1,'%Y-%m-%d') - dt.timedelta(days=28))].copy()
    df_avail_last_month = df_avail_last_month.groupby('article_id').size().reset_index().rename(\
                                                                    columns={0:'PurchaseRateLastMonth'})
    df_avail_last_month['PurchaseRateLastMonth'] = df_avail_last_month['PurchaseRateLastMonth']/4
    df_baseline = pd.merge(df_baseline,df_avail_last_month,on='article_id',how='left').fillna(0)
    df_baseline['LastMonthPopularity'] = np.where(df_baseline['PurchaseRatePerWeek'] == 0,0,\
                                    df_baseline['PurchaseRateLastMonth']/df_baseline['PurchaseRatePerWeek'])
    del df_avail_last_month
    
    return df_baseline

In [ ]:
# Create the train and test sets

print('Sample',sample)
start = dt.datetime.now()
print(start)

for dates in [('2020-05-13','2020-05-19'),('2020-05-20','2020-05-26'),('2020-05-27','2020-06-02'),\
              ('2020-06-03','2020-06-09'),('2020-06-10','2020-06-16'),('2020-06-17','2020-06-23'),\
              ('2020-06-24','2020-06-30'),('2020-07-01','2020-07-07'),('2020-07-08','2020-07-14'),\
              ('2020-07-15','2020-07-21'),('2020-07-22','2020-07-28'),('2020-07-29','2020-08-04'),\
              ('2020-08-05','2020-08-11'),('2020-08-12','2020-08-18'),('2020-08-19','2020-08-25'),\
              ('2020-08-26','2020-09-01'),('2020-09-02','2020-09-08'),('2020-09-09','2020-09-15'),\
              ('2020-09-16','2020-09-22'),('2020-09-23','2020-09-29')]:
    gt = (False if dates[0] == '2020-09-23' else True)
    date_range = ('FULL' if dates[0] == '2020-09-23' else \
                  dates[0][-5:-3]+dates[0][-2:]+'_'+dates[1][-5:-3]+dates[1][-2:])
    print(dates,date_range,dt.datetime.now()-start)
    
    df_train_set = build_dataset(dates[0],dates[1],generate_test = gt)
    df_train_set.to_feather('../Datasets/Repeat'+sample+'/Repurchase_'+date_range+'.feather')